In [21]:
from copy import deepcopy

%run "C:\Users\tomas/Downloads\OA_project_20221894\base\dataset_generation.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\base\individual.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\base\population.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\operators_utils\selection_algorithms.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\operators_utils\crossovers.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\operators_utils\mutators.ipynb"
%run "C:\Users\tomas\Downloads\OA_project_20221894\operators_utils\utils.ipynb"

In [14]:
def GA(matrix, initializer, evaluator, selector, crossover, mutator, elite_func,
       pop_size, n_gens, p_xo, p_m, verbose=False,
       log_path=None, elitism=True, seed=0):
    """
    Perform a Genetic Algorithm (GA) optimization.

    Args:
        matrix (list): A matrix of Geo earnings.
        initializer (function): A function to initialize the population.
        evaluator (function): A function to evaluate the fitness of individuals in the population.
        selector (function): A function to select individuals from the population.
        crossover (function): A function to perform crossover between selected individuals.
        mutator (function): A function to mutate individuals in the population.
        elite_func (function): A function to select elite individuals from the population.
        pop_size (int): Size of the population.
        n_gens (int): Number of generations.
        p_xo (float): Probability of crossover.
        p_m (float): Probability of mutation.
        verbose (bool, optional): Whether to print verbose output (default is False).
        log_path (str, optional): Path to the log file (default is None).
        elitism (bool, optional): Whether to use elitism (default is True).
        seed (int, optional): Seed for random number generation (default is 0).

    Returns:
        tuple: A tuple containing the final population and their fitness values.
    """

    # Setting the seed
    random.seed(seed)

    if elite_func is None:
        raise Exception('elite_func cannot be None')

    # Initializing our population
    population = initializer()(pop_size)

    # Evaluating it
    pop_fit = evaluator(matrix)(population)

    # Initiating our generations
    for gen in range(1, n_gens):

        offspring = []

        # While our offsprings are not of the size of our population, we select two parents to input there
        while len(offspring) < pop_size:
            p1, p2 = selector(population, pop_fit), selector(population, pop_fit)

            # xover or reproduction
            if random.random() < p_xo:
                # do xover
                o1, o2 = crossover(p1,p2)

            else:
                # do reproduc
                o1, o2 = deepcopy(p1), deepcopy(p2)

            o1, o2 = mutator(o1, p_m), mutator(o2,p_m)

            offspring.extend([o1, o2])

        # While our offsprings are longer than the population, we pop the last individual out
        while len(offspring) > pop_size:
            offspring.pop()

        if elitism:
            offspring[- len(elite_func(population, pop_fit)[0]):], best_fits = elite_func(population, pop_fit)

        # Our offsprings turns into our population
        population = offspring
        pop_fit = evaluator(matrix)(population)

        new_elite, new_fit = elite_func(population, pop_fit)[0][-1], elite_func(population, pop_fit)[1][-1]

        if verbose:
                print(f' {gen} | {new_fit} ')
                print('-' * 32)

        if log_path is not None:
             with open(log_path, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([seed, gen, new_fit, new_elite])


    return population, pop_fit

In [24]:
selects = [roulette_wheel, ranking_selection, tournament_selection(2), tournament_selection(5), tournament_selection(10)]
xovers = [cyclic_xover, pmx_xover, ordered_xover, two_point_xover]
mutats = [swap_mutation, inversion_mutation, mix_mutation, disp_mutation, two_point_mutation]

initializer = create_population

evaluator = evaluate_pop

selector = random.choice(selects)

crossover = random.choice(xovers)

mutator = random.choice(mutats)

pop_size = 50

n_gens=200

p_xo=0.7

p_m=0.3

n_elites = 3


GA(matrix=sample_matrix,
   initializer=create_population,
   evaluator=evaluate_pop,
   selector=selector,
   crossover=crossover,
   mutator=mutator,
   pop_size=pop_size,
   n_gens=n_gens,
   p_xo=p_xo,
   p_m=p_m,
   elitism=True,
   elite_func=get_n_elites(n_elites)
  )
   

([[0, 3, 8, 9, 1, 4, 7, 5, 2, 6, 0],
  [0, 5, 3, 1, 8, 9, 2, 4, 6, 7, 0],
  [0, 1, 2, 3, 4, 9, 7, 5, 8, 6, 0],
  [0, 2, 3, 9, 4, 1, 7, 8, 6, 5, 0],
  [0, 6, 2, 4, 3, 7, 1, 5, 8, 9, 0],
  [0, 2, 3, 4, 1, 9, 7, 5, 8, 6, 0],
  [0, 2, 3, 1, 8, 9, 5, 7, 4, 6, 0],
  [0, 1, 8, 7, 6, 9, 5, 3, 4, 2, 0],
  [0, 3, 7, 2, 4, 5, 1, 8, 9, 6, 0],
  [0, 3, 7, 2, 4, 5, 6, 8, 1, 9, 0],
  [0, 2, 3, 9, 5, 1, 8, 4, 7, 6, 0],
  [0, 2, 3, 1, 8, 9, 5, 4, 6, 7, 0],
  [0, 8, 3, 2, 9, 6, 5, 7, 4, 1, 0],
  [0, 2, 3, 1, 8, 9, 7, 5, 6, 4, 0],
  [0, 5, 7, 3, 2, 1, 6, 8, 9, 4, 0],
  [0, 7, 9, 8, 1, 2, 3, 6, 4, 5, 0],
  [0, 1, 9, 5, 6, 3, 2, 7, 8, 4, 0],
  [0, 6, 2, 3, 7, 9, 5, 1, 8, 4, 0],
  [0, 2, 3, 1, 7, 9, 6, 8, 4, 5, 0],
  [0, 2, 3, 1, 8, 9, 5, 7, 4, 6, 0],
  [0, 2, 3, 1, 8, 9, 7, 5, 4, 6, 0],
  [0, 2, 3, 8, 1, 5, 7, 9, 4, 6, 0],
  [0, 9, 8, 2, 3, 1, 5, 4, 7, 6, 0],
  [0, 8, 9, 7, 4, 1, 5, 3, 2, 6, 0],
  [0, 2, 9, 3, 4, 1, 5, 8, 7, 6, 0],
  [0, 3, 7, 2, 1, 5, 9, 8, 6, 4, 0],
  [0, 7, 5, 4, 2, 6, 9, 1, 3, 8, 0],
 